In [1]:
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tqdm import tqdm
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import win32api,win32con

# 模板、ptsku、输出文件路径
template_path = './模板.xlsx'
ptsku_path = erp_data_path = list(os.walk('./数据源'))[0][0] + '/' + list(os.walk('./数据源'))[0][2][0]
output_path = './查重结果.xlsx'

In [2]:
ptsku_erp = pd.read_excel(ptsku_path, dtype=str).fillna('')
result = pd.read_excel('./查重结果.xlsx', dtype=str).fillna('')
result

,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号,计数-完整表内重复辨识号,系统重复OE,重复PTSKU,是否已开发
0,YT-SKU-0001,9004125;9004129,9004125;9004129,9004129,YT-SKU-0001;YT-SKU-1635,YT-SKU-0001;YT-SKU-1635,9004129,2,,,否，表内重复
1,YT-SKU-0002,23416973,23416973,,,,,-,,,否
2,YT-SKU-0003,L26670601,L26670601,,,,,-,,,否
3,YT-SKU-0004,9004130;9004128;9004126,9004130;9004128;9004126,,,,,-,,,否
4,YT-SKU-0005,24114962,24114962,,,,,-,,,否
...,...,...,...,...,...,...,...,...,...,...,...
3104,YT-SKU-3105,2045018282,2045018282,2045018282,YT-SKU-3040;YT-SKU-3105,YT-SKU-2171;YT-SKU-3040;YT-SKU-3105,2045019682;2045018282,3,,,否，表内重复
3105,YT-SKU-3106,2045018782,2045018782,2045018782,YT-SKU-2735;YT-SKU-3106,YT-SKU-2735;YT-SKU-3106,2045018782,2,,,否，表内重复
3106,YT-SKU-3107,2045019482,2045019482,,,,,-,2045019482,TH19482-FC,是
3107,YT-SKU-3108,1665280082,1665280082,,,,,-,1665280082,TH03362-FC,是


In [4]:
# need_ptsku_data1 = ptsku_erp['SKU', '查重用OE', '品类简称']
need_ptsku_data2 = ptsku_erp[['SKU', '供应商简称', '最新价格（含税）']]

In [5]:
need_ptsku_data2

,SKU,供应商简称,最新价格（含税）
0,MGS61S21-C,台州金日,26.0000
1,MGS21D30-C,台州金日,26.0000
2,MGS61S20-C,台州金日,26.0000
3,MGS762S8-C,台州金日,27.0000
4,MGS11AA0-C,台州金日,26.0000
...,...,...,...
167773,SGAU002,台州拓野,48.6400
167774,SGAU001,台州拓野,48.6400
167775,SGAC005,台州拓野,50.0000
167776,SGAC003,台州拓野,55.0000


,辨识号,PTSKU_1,PTSKU_2,PTSKU_3,PTSKU_4,PTSKU_5,PTSKU_6,PTSKU_7,PTSKU_8,PTSKU_9
21,YT-SKU-0022,RH37715,None,None,None,None,None,None,None,None
29,YT-SKU-0030,RH37778,None,None,None,None,None,None,None,None
109,YT-SKU-0110,TH02B00-FC,None,None,None,None,None,None,None,None
153,YT-SKU-0154,WCP05210,None,None,None,None,None,None,None,None
154,YT-SKU-0155,RH72159-FC,RH72159-C,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
3100,YT-SKU-3101,RH02906-FC,None,None,None,None,None,None,None,None
3103,YT-SKU-3104,TH12682,None,None,None,None,None,None,None,None
3106,YT-SKU-3107,TH19482-FC,None,None,None,None,None,None,None,None
3107,YT-SKU-3108,TH03362-FC,None,None,None,None,None,None,None,None


In [64]:
# 取重复辨识号不为空部分，分离为单独的PTSKU
need_result = result[['辨识号', '重复PTSKU']]
clean_need_result = need_result[need_result['重复PTSKU']!='']
new_need_result = clean_need_result[['辨识号','重复PTSKU']].copy()
split_cols = new_need_result['重复PTSKU'].str.split(';', expand=True)
# 重命名列索引，逐一合并
split_cols_columns = [f'PTSKU_{int(i)+1}' for i in range(split_cols.shape[1])]
split_cols.columns = split_cols_columns
new_need_result = pd.concat([new_need_result[['辨识号']], split_cols], axis=1)
# 匹配供应商及价格
result_process = new_need_result[['辨识号']]
for i in split_cols_columns:
    index_i = int(split_cols_columns.index(i))
    index_sku = str(index_i+1)
    process_columns = ['辨识号', i]
    df_process = new_need_result[process_columns]
    df_process = pd.merge(df_process, need_ptsku_data2, left_on=i, right_on='SKU', how='left').fillna('')
    df_process = df_process[['辨识号', i, '供应商简称', '最新价格（含税）']]
    df_process.rename(columns={'供应商简称':'供应商_'+index_sku, '最新价格（含税）':'采购价_'+index_sku}, inplace=True)
    result_process = pd.merge(result_process, df_process, on='辨识号', how='left').fillna('')
result_process

,辨识号,PTSKU_1,供应商_1,采购价_1,PTSKU_2,供应商_2,采购价_2,PTSKU_3,供应商_3,采购价_3,...,采购价_6,PTSKU_7,供应商_7,采购价_7,PTSKU_8,供应商_8,采购价_8,PTSKU_9,供应商_9,采购价_9
0,YT-SKU-0022,RH37715,瑞安通达,42.0000,,,,,,,...,,,,,,,,,,
1,YT-SKU-0030,RH37778,瑞安通达,48.0000,,,,,,,...,,,,,,,,,,
2,YT-SKU-0110,TH02B00-FC,瑞安通达,91.0000,,,,,,,...,,,,,,,,,,
3,YT-SKU-0154,WCP05210,清河宝世达,33.9500,,,,,,,...,,,,,,,,,,
4,YT-SKU-0155,RH72159-FC,清河宝世达,24.2500,RH72159-C,清河宝世达,24.2500,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,YT-SKU-3101,RH02906-FC,青岛建邦,85.0000,,,,,,,...,,,,,,,,,,
512,YT-SKU-3104,TH12682,瑞安通达,40.0000,,,,,,,...,,,,,,,,,,
513,YT-SKU-3107,TH19482-FC,瑞安通达,48.0000,,,,,,,...,,,,,,,,,,
514,YT-SKU-3108,TH03362-FC,瑞安通达,98.0000,,,,,,,...,,,,,,,,,,


In [28]:
# 分割 column_name 列的值
split_cols = df['column_name'].str.split(';', expand=True)

# 将分割后的列与原 DataFrame 连接
df = pd.concat([df, split_cols], axis=1)

# 可选：重命名新的列名（可选的）
split_cols.columns = [f'col_{i}' for i in range(split_cols.shape[1])]
df = pd.concat([df.drop(columns='column_name'), split_cols], axis=1)

df

,0,1,2,3,col_0,col_1,col_2,col_3
0,a,b,c,None,a,b,c,None
1,d,e,None,None,d,e,None,None
2,f,g,h,i,f,g,h,i
